In [0]:
import pickle
import json
import datetime
import os

deployment_dir = "/Volumes/workspace/default/project/"
model_path = deployment_dir +  "models/rf_model.pkl"

# Test model loading
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)
print(" Model loading test successful")


✅ Model loading test successful


In [0]:
print("=== CREATING FASTAPI APPLICATION ===")

fastapi_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pickle
import pandas as pd
import numpy as np
import json
from typing import List, Dict
import os

# Initialize FastAPI app
app = FastAPI(
    title="Customer Repeat Purchase Predictor",
    description="ML API for predicting customer repeat purchases in next 30 days",
    version="1.0.0"
)

# Global model variables
model = None
model_metadata = None

@app.on_event("startup")
async def load_model():
    """Load model and metadata on startup"""
    global model, model_metadata
    
    # Load model
    model_path = "models/rf_model.pkl"
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # Load metadata
    metadata_path = "models/model_metadata.json"
    with open(metadata_path, 'r') as f:
        model_metadata = json.load(f)
    
    print(" Model loaded successfully")

class CustomerFeatures(BaseModel):
    """Input schema for customer features"""
    avg_order_value: float
    purchase_frequency: int
    total_lifetime_value: float
    customer_tenure_days: int
    recency_score: float
    spending_velocity: float
    order_value_coefficient_variation: float
    last_order_month: int
    last_order_day_of_week: int
    is_weekend: int
    is_holiday_season: int
    preferred_category_encoded: int
    preferred_payment_type_encoded: int
    preferred_delivery_location_encoded: int
    # Add dummy features with default values
    category_groceries: int = 0
    category_electronics: int = 0
    category_clothing: int = 0
    category_home_appliances: int = 0
    category_stationery: int = 0
    payment_credit_card: int = 0
    payment_digital_wallet: int = 0
    payment_bank_transfer: int = 0
    payment_cash: int = 0
    location_lalitpur: int = 0
    location_biratnagar: int = 0
    location_bhaktapur: int = 0

class PredictionResponse(BaseModel):
    """Output schema for predictions"""
    customer_id: str
    prediction: int
    probability: float
    confidence: str
    model_version: str

@app.get("/")
async def root():
    """Health check endpoint"""
    return {
        "message": "Customer Repeat Purchase Predictor API",
        "status": "healthy",
        "model_version": model_metadata["version"] if model_metadata else "unknown"
    }

@app.get("/model/info")
async def model_info():
    """Get model information"""
    if model_metadata is None:
        raise HTTPException(status_code=500, detail="Model not loaded")
    
    return {
        "model_name": model_metadata["model_name"],
        "model_type": model_metadata["model_type"],
        "version": model_metadata["version"],
        "performance": model_metadata["performance"],
        "features_count": len(model_metadata["features"])
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict_repeat_purchase(features: CustomerFeatures, customer_id: str = "unknown"):
    """Predict repeat purchase probability for a customer"""
    
    if model is None:
        raise HTTPException(status_code=500, detail="Model not loaded")
    
    try:
        # Convert input to DataFrame
        feature_dict = features.dict()
        input_df = pd.DataFrame([feature_dict])
        
        # Ensure feature order matches training data
        expected_features = model_metadata["features"]
        input_df = input_df.reindex(columns=expected_features, fill_value=0)
        
        # Make prediction
        prediction = model.predict(input_df)[0]
        probability = model.predict_proba(input_df)[0][1]
        
        # Determine confidence level
        if probability >= 0.7:
            confidence = "high"
        elif probability >= 0.4:
            confidence = "medium"
        else:
            confidence = "low"
        
        return PredictionResponse(
            customer_id=customer_id,
            prediction=int(prediction),
            probability=round(float(probability), 4),
            confidence=confidence,
            model_version=model_metadata["version"]
        )
        
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Prediction error: {str(e)}")

@app.post("/predict/batch")
async def predict_batch(customers: List[Dict]):
    """Batch prediction for multiple customers"""
    
    if model is None:
        raise HTTPException(status_code=500, detail="Model not loaded")
    
    try:
        # Convert to DataFrame
        input_df = pd.DataFrame(customers)
        
        # Ensure feature order
        expected_features = model_metadata["features"]
        input_df = input_df.reindex(columns=expected_features, fill_value=0)
        
        # Make predictions
        predictions = model.predict(input_df)
        probabilities = model.predict_proba(input_df)[:, 1]
        
        results = []
        for i, (pred, prob) in enumerate(zip(predictions, probabilities)):
            confidence = "high" if prob >= 0.7 else "medium" if prob >= 0.4 else "low"
            
            results.append({
                "customer_index": i,
                "prediction": int(pred),
                "probability": round(float(prob), 4),
                "confidence": confidence
            })
        
        return {
            "predictions": results,
            "model_version": model_metadata["version"],
            "batch_size": len(customers)
        }
        
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Batch prediction error: {str(e)}")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# Save FastAPI application
api_file_path = f"{deployment_dir}/api/main.py"
with open(api_file_path, 'w') as f:
    f.write(fastapi_code)

print(f" FastAPI application created: {api_file_path}")

=== CREATING FASTAPI APPLICATION ===
✅ FastAPI application created: /Volumes/workspace/default/project//api/main.py


In [0]:
print("=== CREATING DEPLOYMENT CONFIGURATION ===")

# Requirements file
requirements = '''
fastapi==0.104.1
uvicorn==0.24.0
pydantic==2.5.0
scikit-learn==1.3.2
pandas==2.1.4
numpy==1.24.3
python-multipart==0.0.6
'''

requirements_path = f"{deployment_dir}/requirements.txt"
with open(requirements_path, 'w') as f:
    f.write(requirements)

# Dockerfile
dockerfile_content = '''
FROM python:3.9-slim

WORKDIR /app

# Copy requirements and install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code and model
COPY api/ ./
COPY models/ ./models/

# Expose port
EXPOSE 8000

# Run the application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
'''

dockerfile_path = f"{deployment_dir}/Dockerfile"
with open(dockerfile_path, 'w') as f:
    f.write(dockerfile_content)

# Docker Compose for easy deployment
docker_compose = '''
version: '3.8'

services:
  ml-api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - MODEL_ENV=production
    volumes:
      - ./models:/app/models:ro
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/"]
      interval: 30s
      timeout: 10s
      retries: 3
'''

compose_path = f"{deployment_dir}/docker-compose.yml"
with open(compose_path, 'w') as f:
    f.write(docker_compose)

print(f" Requirements.txt created: {requirements_path}")
print(f" Dockerfile created: {dockerfile_path}")
print(f" Docker Compose created: {compose_path}")

=== CREATING DEPLOYMENT CONFIGURATION ===
✅ Requirements.txt created: /Volumes/workspace/default/project//requirements.txt
✅ Dockerfile created: /Volumes/workspace/default/project//Dockerfile
✅ Docker Compose created: /Volumes/workspace/default/project//docker-compose.yml


In [0]:
print("=== CREATING TEST EXAMPLES ===")

# Sample test data based on your actual features
test_example = {
    "avg_order_value": 25000.50,
    "purchase_frequency": 3,
    "total_lifetime_value": 75000.0,
    "customer_tenure_days": 90,
    "recency_score": 0.015,
    "spending_velocity": 833.33,
    "order_value_coefficient_variation": 0.2,
    "last_order_month": 7,
    "last_order_day_of_week": 3,
    "is_weekend": 0,
    "is_holiday_season": 0,
    "preferred_category_encoded": 1,
    "preferred_payment_type_encoded": 2,
    "preferred_delivery_location_encoded": 1,
    "category_groceries": 1,
    "payment_credit_card": 1,
    "location_lalitpur": 1
}

# Test script
test_script = f'''
import requests
import json

# API endpoint (update with your deployment URL)
API_URL = "http://localhost:8000"

# Test data
test_data = {json.dumps(test_example, indent=2)}

# Test single prediction
def test_single_prediction():
    url = f"{{API_URL}}/predict"
    
    response = requests.post(
        url,
        json=test_data,
        params={{"customer_id": "CUST_12345"}}
    )
    
    if response.status_code == 200:
        result = response.json()
        print(" Single Prediction Success:")
        print(f"Customer ID: {{result['customer_id']}}")
        print(f"Prediction: {{result['prediction']}}
        print(f"Probability: {{result['probability']:.3f}}")
        print(f"Confidence: {{result['confidence']}}")
    else:
        print(f" Error: {{response.status_code}} - {{response.text}}")

# Test batch prediction
def test_batch_prediction():
    url = f"{{API_URL}}/predict/batch"
    
    batch_data = [test_data for _ in range(3)]  # 3 identical customers for testing
    
    response = requests.post(url, json=batch_data)
    
    if response.status_code == 200:
        result = response.json()
        print("\\n Batch Prediction Success:")
        print(f"Batch size: {{result['batch_size']}}")
        for pred in result['predictions']:
            print(f"Customer {{pred['customer_index']}}: {{pred['prediction']}} (prob: {{pred['probability']}})")
    else:
        print(f" Batch Error: {{response.status_code}} - {{response.text}}")

# Test model info
def test_model_info():
    url = f"{{API_URL}}/model/info"
    response = requests.get(url)
    
    if response.status_code == 200:
        info = response.json()
        print("\\n Model Info:")
        print(f"Model: {{info['model_name']}}")
        print(f"Type: {{info['model_type']}}")
        print(f"Version: {{info['version']}}")
        print(f"F1-Score: {{info['performance']['test_f1_score']:.3f}}")
    else:
        print(f" Info Error: {{response.status_code}}")

if __name__ == "__main__":
    print(" Testing Customer Repeat Purchase Predictor API")
    
    # Test endpoints
    test_model_info()
    test_single_prediction()
    test_batch_prediction()
    
    print("\\n All tests completed!")
'''

test_script_path = f"{deployment_dir}/test_api.py"
with open(test_script_path, 'w') as f:
    f.write(test_script)

print(f" Test script created: {test_script_path}")

=== CREATING TEST EXAMPLES ===
✅ Test script created: /Volumes/workspace/default/project//test_api.py
